---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import re
    from collections import defaultdict
    uni_dict= defaultdict(list)

    with open('university_towns.txt','r') as uni_towns:
        for lines in uni_towns:
            if "edit" in lines:
                mykey= re.match(r'.+?(?=\[edit])',lines).group()
            
            elif ':' in lines:
                myvalues = re.match(r'.+?(?=:)',lines).group()
                uni_dict[mykey].append(myvalues)
            
            elif '(' in lines:
                myvalues= re.match(r'.+?(?=\()',lines).group()
                myvalues=myvalues[:-1]
                uni_dict[mykey].append(myvalues)
            else:
                myvalues=lines
                uni_dict[mykey].append(myvalues)
        
                

    list_stuff=[]
    for k,v in uni_dict.items():
        for values in v:
            list_stuff.append(k + ',' + values )
    state=[]
    county=[]
    for uni in list_stuff:
        s=uni.split(',')[0]
        c=uni.split(',')[1]
        state.append(s)
        county.append(c)
    df1=pd.DataFrame(state,columns=['state'])
    df2=pd.DataFrame(county,columns=['county'])
    df_uni=df1.join(df2)
    return df_uni
#get_list_of_university_towns()

In [18]:
df_quaterlyGDP = pd.read_excel('gdplev.xls',usecols=[4,6],skiprows=220,header=None,
             names=['Quarter','GDP in billions of chained 2009 dollars'])

df_quaterlyGDP['pct_change'] = df_quaterlyGDP['GDP in billions of chained 2009 dollars'].astype(float).pct_change(1)
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    recession_start=df_quaterlyGDP.loc[(df_quaterlyGDP['pct_change']<0) & 
                                       (df_quaterlyGDP['pct_change'].shift(-1)<0) & (df_quaterlyGDP['pct_change'].shift(1)>0)]#.head(1).Quarter.values[0]
    
    return recession_start
get_recession_start()


,Quarter,GDP in billions of chained 2009 dollars,pct_change
34,2008q3,14891.6,-0.004798


In [24]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_end=df_quaterlyGDP.loc[(df_quaterlyGDP['pct_change']<0) & 
                                     (df_quaterlyGDP['pct_change'].shift(-1)>0) &
                                     (df_quaterlyGDP['pct_change'].shift(1)<0) &
                                    (df_quaterlyGDP['pct_change'].shift(-2)>0)].head(1).Quarter.values[0]
       
    return recession_end
get_recession_end()

'2009q2'

In [25]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_start=df_quaterlyGDP.loc[(df_quaterlyGDP['pct_change']<0) & 
                                       (df_quaterlyGDP['pct_change'].shift(-1)<0)].head(1).Quarter.values[0]
    
    recession_end=df_quaterlyGDP.loc[(df_quaterlyGDP['pct_change']<0) & 
                                     (df_quaterlyGDP['pct_change'].shift(-1)>0) &
                                     (df_quaterlyGDP['pct_change'].shift(1)<0) &
                                    (df_quaterlyGDP['pct_change'].shift(2)<0)].head(1).Quarter.values[0]
    
    rec_start=df_quaterlyGDP.loc[df_quaterlyGDP['Quarter']==recession_start].index.values[0]
    rec_end=df_quaterlyGDP.loc[df_quaterlyGDP['Quarter']==recession_end].index.values[0]
    
    recession_bottom=df_quaterlyGDP.iloc[rec_start:rec_end+1].sort_values('pct_change').head(1).Quarter.values[0]

    return recession_bottom
get_recession_bottom()

'2008q4'

In [27]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_house=pd.read_csv('City_Zhvi_AllHomes.csv')
    df_house.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1,inplace=True)
    df_house.set_index(['State','RegionName'],inplace=True)
    cols=range(0,45)
    df_house.drop(df_house.columns[cols],axis=1,inplace=True)
    df_quart=df_house.groupby(np.arange(len(df_house.columns))//3,axis=1).mean()
    
    def column_name():
        col_name=[]
        dic={}
        years=list(range(2000,2017))
        quarter=['q1','q2','q3','q4']
        for y in years:
            for q in quarter:
                col_name.append(str(y)+q)
        col_name=col_name[:-1]
        for old,new in zip(df_quart.columns,col_name):
            dic[old]=new
        return dic
    dictionary=column_name()
    
    pd.set_option('display.float_format', lambda x: '%.3f' % x)

    df_quart.rename(columns=dictionary,inplace=True)
    
    return df_quart
#convert_housing_data_to_quarters()

In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df_states=pd.DataFrame.from_dict(states,orient='index')
    df_states=df_states.reset_index().rename(columns={'index':'State',0:'State_name'}).set_index('State_name').sort_index()
    uni_towns=get_list_of_university_towns()
    uni_towns.set_index('state',inplace=True)
    uni_towns=uni_towns.join(df_states,how='inner').rename(columns={'county':'RegionName'})
    uni_towns=uni_towns.reset_index().set_index(['State','RegionName']).drop('index',axis=1)
    uni_towns['uni_town']=True
    df_house=convert_housing_data_to_quarters()
    df_house
    df_uni_price=df_house.reset_index().join(uni_towns,on=['State','RegionName']).set_index(df_house.index.names)
    df_uni_price['uni_town']=df_uni_price['uni_town'].fillna(value=False)

    recession_start=df_quaterlyGDP.loc[(df_quaterlyGDP['pct_change']<0) & 
                                       (df_quaterlyGDP['pct_change'].shift(-1)<0)].head(1).Quarter.values[0]
    
    recession_bottom=get_recession_bottom()

    before_recession=(df_uni_price.columns.get_loc(recession_start))-1
    df_uni_price['price_ratio']=df_uni_price.iloc[:,before_recession]/df_uni_price.loc[:,recession_bottom]

    pvalue=ttest_ind(df_uni_price[df_uni_price['uni_town']==True]['price_ratio'],
          df_uni_price[df_uni_price['uni_town']==False]['price_ratio'],nan_policy='omit').pvalue
    better=df_uni_price.groupby(['uni_town'])['price_ratio'].agg({'avg':np.mean}).sort_values('avg').head(1).index.values[0]

    if pvalue < 0.01:
        if better == True:
            ans=(True, + pvalue, 'university_town')
        else:
            ans=(True,pvalue,'non-university_town')
    else:
        if better == True:
            ans=(False, + pvalue, 'university_town')
        else:
            ans=(False,pvalue,'non-university_town')
    
    return ans
run_ttest()

(True, 0.0010936185907365866, 'university_town')